# Step 1

## Install Dependensi

In [1]:
!nvidia-smi

Fri Dec 27 18:05:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0              67W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers

In [3]:
!nvidia-smi
!echo "bitsandbytes version:"
!pip show bitsandbytes
!echo "peft version:"
!pip show peft
!echo "trl version:"
!pip show trl
!echo "accelerate version:"
!pip show accelerate
!echo "datasets version:"
!pip show datasets
!echo "transformers version:"
!pip show transformers


Fri Dec 27 18:05:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

Tentukan ID Hugging Face untuk load model dan menyimpan model.

In [5]:
# Hugging Face model hub identifier for the model V.1

# import os
# from google.colab import userdata
# os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [45]:
# Hugging Face model hub identifier for the model V.2

from huggingface_hub import notebook_login
notebook_login()

# Step 2 - Model loading
Muat model menggunakan kuantisasi QLoRA untuk mengurangi penggunaan memori.


In [7]:
model_id = "GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    a

In [8]:
def get_completion(query: str, model, tokenizer) -> str:
    device = "cuda:0"

    # Template untuk bahasa Indonesia sesuai dengan data
    prompt_template = """
    <start_of_turn>user
    Di bawah ini terdapat instruksi yang menjelaskan sebuah tugas. Tulis respons yang sesuai untuk melengkapi permintaan. ### Instruksi: {query}
    <end_of_turn>
    <start_of_turn>model
    """

    # Format prompt dengan query yang diberikan
    prompt = prompt_template.format(query=query)

    # Tokenisasi prompt
    encodes = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

    # Memindahkan token ke perangkat GPU/CPU
    model_inputs = encodes.to(device)

    # Generasi output dari model
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1000,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode hasil output ke string
    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return decoded

In [9]:
result = get_completion(query="Apa saja layanan kesehatan yang disediakan oleh RSUD dr. H. Moh. Anwar Sumenep?", model=model, tokenizer=tokenizer)
print(result)


    user
    Di bawah ini terdapat instruksi yang menjelaskan sebuah tugas. Tulis respons yang sesuai untuk melengkapi permintaan. ### Instruksi: Apa saja layanan kesehatan yang disediakan oleh RSUD dr. H. Moh. Anwar Sumenep?
    
    model
    RSUD dr. H. Moh. Anwar Sumenep menyediakan berbagai layanan kesehatan, termasuk layanan rawat inap, rawat jalan, poliklinik spesialis, instalasi gawat darurat, laboratorium, radiologi, dan layanan pendukung lainnya. RSUD dr. H. Moh. Anwar Sumenep juga memiliki fasilitas unit-unit khusus seperti ruang perawatan intensif, ruang bersalin, dan perawatan neonatus. Spesialis yang ada meliputi Spesialis Anaesthesiologi & Reanimasi Klinis, Spesialis Bedah Umum, Spesialis Penyakit Dalam, Spesialis Kandungan, Spesialis Anak, Spesialis Kebidanan dan Kandungan, Spesialis Mata, Spesialis Kulit & Kelamin, Spesialis THT, Spesialis Gigi, Spesialis Jantung & Pembuluh Darah, Spesialis Ortopedi. Seluruh layanan kesehatan disediakan sesuai dengan standar dan regul

# Step 3 - Load dataset for finetuning

### Lets Load the Dataset



Kami akan menggunakan dataset ini untuk _fine-tuning_ model. Dataset ini mengikuti gaya instruksi alpaca, yang merupakan titik awal yang sangat baik untuk tugas ini. Struktur dataset seharusnya seperti:

```json
{
   "instruction": "Di mana lokasi RSUD dr. H. Moh. Anwar Sumenep?",
   "input": "Kategori Lokasi",
   "output": "RSUD dr. H. Moh. Anwar Sumenep berlokasi di Jl. Dr. Cipto No. 42, Desa Kolor, Kecamatan Kota Sumenep, Jawa Timur."
}
```

In [10]:
from datasets import load_dataset

dataset = load_dataset("aqilwahid/data_statis", split="train")
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 119
})

In [11]:
df = dataset.to_pandas()
df.head(10)

,instruction,input,output,text
0,Apa layanan kesehatan yang disediakan oleh RSU...,Kategori Medis,RSUD dr. H. Moh. Anwar Sumenep menyediakan lay...,Di bawah ini terdapat instruksi yang menjelask...
1,Di mana lokasi RSUD dr. H. Moh. Anwar Sumenep?,Kategori Lokasi,RSUD dr. H. Moh. Anwar Sumenep berlokasi di Jl...,Di bawah ini terdapat instruksi yang menjelask...
2,Apa visi RSUD dr. H. Moh. Anwar Sumenep?,Kategori Umum,Visi RSUD dr. H. Moh. Anwar Sumenep adalah men...,Di bawah ini terdapat instruksi yang menjelask...
3,Bagaimana cara masyarakat mendapatkan layanan ...,Kategori Medis,Masyarakat dapat memanfaatkan program Universa...,Di bawah ini terdapat instruksi yang menjelask...
4,Apa saja fasilitas unggulan yang dimiliki oleh...,Kategori Medis,RSUD dr. H. Moh. Anwar Sumenep memiliki fasili...,Di bawah ini terdapat instruksi yang menjelask...
5,Siapa direktur RSUD dr. H. Moh. Anwar Sumenep ...,Kategori Direktori,Direktur RSUD dr. H. Moh. Anwar Sumenep saat i...,Di bawah ini terdapat instruksi yang menjelask...
6,Apa saja layanan yang tersedia di Poliklinik M...,Kategori Medis,Poliklinik Mata RSUD dr. H. Moh. Anwar Sumenep...,Di bawah ini terdapat instruksi yang menjelask...
7,Bagaimana sejarah pendirian RSUD dr. H. Moh. A...,Kategori Umum,RSUD dr. H. Moh. Anwar Sumenep didirikan pada ...,Di bawah ini terdapat instruksi yang menjelask...
8,Apa saja program inovatif yang telah dilakukan...,Kategori Umum,RSUD dr. H. Moh. Anwar Sumenep telah melakukan...,Di bawah ini terdapat instruksi yang menjelask...
9,Bagaimana cara masyarakat memberikan masukan a...,Kategori Medis,Masyarakat dapat memberikan masukan atau penga...,Di bawah ini terdapat instruksi yang menjelask...


In [12]:
# Fungsi untuk menghasilkan prompt berformat gemma2

def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenized prompt
    """
    prefix_text = 'Di bawah ini terdapat instruksi yang menjelaskan sebuah tugas. Tulis respons yang sesuai untuk melengkapi permintaan.\n\n'

    # Samples with additional context info
    if data_point['input']:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} Berikut adalah inputnya: {data_point["input"]} <end_of_turn>\n<start_of_turn>model {data_point["output"]} <end_of_turn>"""
    # Without additional context
    else:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model {data_point["output"]} <end_of_turn>"""

    return text

In [13]:
# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

In [14]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 119
})

In [15]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [16]:
print(test_data)

Dataset({
    features: ['instruction', 'input', 'output', 'text', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 24
})


# Step 4 - Apply Lora  
PeftModel dan gunakan _low-rank adapters_ (LoRA) menggunakan _get_peft_model utility function_ dan metode _prepare_model_for_kbit_training_ dari PEFT.

In [17]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [18]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [19]:
modules = find_all_linear_names(model)
print(modules)

['k_proj', 'q_proj', 'gate_proj', 'up_proj', 'v_proj', 'o_proj', 'down_proj']


In [20]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [21]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 216072192 | total: 9457778176 | Percentage: 2.2846%


# Step 5 - Run the training!

## With HF

In [22]:
import transformers

tokenizer.pad_token = tokenizer.eos_token


trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=3,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs_mistral_b_finance_finetuned_test",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


### Fine-Tuning with qLora and Supervised Fine-Tuning

Berikutnya _fine-tuning_ model menggunakan qLora. Kita menggunakan `SFTTrainer` dari pustaka `trl` untuk _fine-tuning_ _Supervised_. Pastikan telah menginstal pustaka `trl`.

In [23]:
# import transformers

# from trl import SFTTrainer


# tokenizer.pad_token = tokenizer.eos_token
# torch.cuda.empty_cache()


# trainer = SFTTrainer(
#     model=model,
#     train_dataset=train_data,
#     eval_dataset=test_data,
#     dataset_text_field="prompt",
#     peft_config=lora_config,
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=1,
#         gradient_accumulation_steps=4,
#         #warmup_steps=0.03,
#         max_steps=100,
#         learning_rate=2e-4,
#         logging_steps=1,
#         output_dir="outputs",
#         optim="paged_adamw_8bit",
#         save_strategy="epoch",
#     ),
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
# )

## Lets start training

In [24]:
from accelerate.state import AcceleratorState

# Reset accelerator state
if AcceleratorState._shared_state:
    AcceleratorState._shared_state = {}


In [25]:
from accelerate import Accelerator

# Inisialisasi accelerator
accelerator = Accelerator()
trainer.accelerator = accelerator


In [26]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: aqilwahid (aqilwahid-ai). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,3.765500
2,2.914200
3,3.342600
4,2.487000
5,1.957300
6,1.616900
7,1.542100
8,1.453400
9,1.698800
10,1.365500


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

TrainOutput(global_step=100, training_loss=0.6152884207665921, metrics={'train_runtime': 371.6372, 'train_samples_per_second': 1.076, 'train_steps_per_second': 0.269, 'total_flos': 2849445398274048.0, 'train_loss': 0.6152884207665921, 'epoch': 4.168421052631579})

 Share adapters on the 🤗 Hub

In [27]:
new_model = "RSUD-MA-Fine-Tune-Gemma2" # Nama model yang akan di push ke Hugging Face

In [28]:
trainer.model.save_pretrained(new_model)

In [29]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Test out Finetuned Model

In [36]:
result = get_completion(query="Apa layanan kesehatan yang disediakan oleh RSUD dr. H. Moh. Anwar Sumenep?", model=merged_model, tokenizer=tokenizer)

In [41]:
result

'\n    user\n    Di bawah ini terdapat instruksi yang menjelaskan sebuah tugas. Tulis respons yang sesuai untuk melengkapi permintaan. ### Instruksi: Apa layanan kesehatan yang disediakan oleh RSUD dr. H. Moh. Anwar Sumenep?\n    \n    model\n    \nmodel Layanan kesehatan yang disediakan meliputi: 1) Penanganan penyakit non-menular seperti jantung, stroke, dan diabetes, 2) Penanganan penyakit menular seperti HIV/AIDS dan tuberkulosis, 3) Pelayanan kesehatan lansia dan penyakit degeneratif, 4) Pelayanan kesehatan jiwa, 5) K3RS (Konsultasi, Pengobatan, dan Rekomendasi Senam) untuk pasien rehabilitaasi medis, dan 6) Pelayanan kesehatan gigi dengan satu dokter gigi spesialis. \nmodel \nmodel \nmodel Kategori layanan: 1. Rawat Jalan 2. Rawat Inap \nmodel \nmodel Kategori Medis: **A. Spesialis** \n</div>\nmodel \nmodel Kategori Medis: **A. Spesialis** \n</div>model RSUD dr. H. Moh. Anwar Sumenep memiliki 14 ruang perawatan rawat inap dengan kapasias masing-masing 2-4 tempat tidur. Ketersedia

## Push to Hugging Face

In [46]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aqilwahid/RSUD-MA-Fine-Tune-Gemma2/commit/f0356081df87f0eac8a8db3c90d24a9546abb231', commit_message='Upload tokenizer', commit_description='', oid='f0356081df87f0eac8a8db3c90d24a9546abb231', pr_url=None, repo_url=RepoUrl('https://huggingface.co/aqilwahid/RSUD-MA-Fine-Tune-Gemma2', endpoint='https://huggingface.co', repo_type='model', repo_id='aqilwahid/RSUD-MA-Fine-Tune-Gemma2'), pr_revision=None, pr_num=None)